In [149]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from sklearn.metrics.pairwise import cosine_similarity

pd.options.display.max_rows = 10
pd.options.display.max_columns = 15

guest_id = int(input('insert a user id : '))

In [150]:
data = pd.read_table('data/u.data', sep='\t', usecols=['user id', 'movie id', 'rating'])
data

,user id,movie id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [151]:
data = data.pivot_table(index='movie id', columns='user id', values='rating', fill_value=0)
data

user id,1,2,3,4,5,6,7,...,937,938,939,940,941,942,943
movie id,,,,,,,,,,,,,,,
1,5,4,0,0,4,4,0,...,0,4,0,0,5,0,0
2,3,0,0,0,3,0,0,...,0,0,0,0,0,0,5
3,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0
4,3,0,0,0,0,0,5,...,0,0,0,2,0,0,0
5,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0
1679,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0
1680,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0


In [152]:
sim = DataFrame()
for i in data.columns:
    sim[i] = DataFrame(cosine_similarity([data[guest_id]], [data[i]]))

top10_near = sim.T.sort_values(by=0, ascending=False).head(11).T
top10_near

,144,592,533,178,271,293,92,474,207,457,62
0,1.0,0.537886,0.489745,0.48299,0.482108,0.480668,0.480351,0.478395,0.475969,0.475433,0.467045


In [153]:
drop_df = data[data[guest_id] > 0][guest_id].to_frame('rating').T
drop_df

movie id,1,4,7,8,9,12,14,...,1147,1169,1197,1226,1284,1285,1286
rating,4,4,2,4,5,4,4,...,4,4,4,4,3,3,4


In [154]:
mult_rec = DataFrame()
for i in top10_near.columns:
    if i != guest_id:
        mult_rec[i] = data[i] * (1 + top10_near[i][0])

data = mult_rec.drop(drop_df)
data

,592,533,178,271,293,92,474,207,457,62
movie id,,,,,,,,,,
2,0.000000,0.00000,5.931959,1.482108,4.442005,4.441054,0.0,4.427907,0.0,0.000000
3,6.151545,0.00000,0.000000,0.000000,2.961336,0.000000,0.0,2.951938,0.0,4.401135
5,0.000000,0.00000,0.000000,0.000000,4.442005,5.921406,0.0,4.427907,0.0,0.000000
6,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
10,0.000000,2.97949,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...
1678,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
1679,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
1680,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000


In [155]:
data = data.T.sum().sort_values(ascending=False).to_frame('recommend rate').head(10)
data

,recommend rate
movie id,
134,60.931253
28,59.414830
238,56.452162
121,56.444851
210,56.220745
265,55.006729
483,55.000876
179,54.974038
203,53.593829


In [156]:
item = pd.read_table('data/u.item', sep='|', usecols=['movie id', 'movie title', 'release date'])
item

,movie id,movie title,release date
0,1,Toy Story (1995),01-Jan-1995
1,2,GoldenEye (1995),01-Jan-1995
2,3,Four Rooms (1995),01-Jan-1995
3,4,Get Shorty (1995),01-Jan-1995
4,5,Copycat (1995),01-Jan-1995
...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998
1678,1679,B. Monkey (1998),06-Feb-1998
1679,1680,Sliding Doors (1998),01-Jan-1998
1680,1681,You So Crazy (1994),01-Jan-1994


In [157]:
item = item.set_index(['movie id'])
item

,movie title,release date
movie id,,
1,Toy Story (1995),01-Jan-1995
2,GoldenEye (1995),01-Jan-1995
3,Four Rooms (1995),01-Jan-1995
4,Get Shorty (1995),01-Jan-1995
5,Copycat (1995),01-Jan-1995
...,...,...
1678,Mat' i syn (1997),06-Feb-1998
1679,B. Monkey (1998),06-Feb-1998
1680,Sliding Doors (1998),01-Jan-1998


In [158]:
pd.concat([data, item], axis=1, join_axes=[data.index])

,recommend rate,movie title,release date
movie id,,,
134,60.931253,Citizen Kane (1941),01-Jan-1941
28,59.414830,Apollo 13 (1995),01-Jan-1995
238,56.452162,Raising Arizona (1987),01-Jan-1987
121,56.444851,Independence Day (ID4) (1996),03-Jul-1996
210,56.220745,Indiana Jones and the Last Crusade (1989),01-Jan-1989
265,55.006729,"Hunt for Red October, The (1990)",01-Jan-1990
483,55.000876,Casablanca (1942),01-Jan-1942
179,54.974038,"Clockwork Orange, A (1971)",01-Jan-1971
203,53.593829,Unforgiven (1992),01-Jan-1992
